In [26]:

from pympler import asizeof
import random
import numpy as np
import time

# Explaining the choice of data structure

In the uncompressed form image can be represented as an array of shape (100000,100000) which is not feasible. Since the array consist of only two different integers 1(for white background) and 0(for black blob), using runlength encoding on each row would be beneficial as it makes the shape linear (from (100000,100000) to (100000,3) (if the row is of the form [1,1,1 ......,0,0,0....,1,1,1]))

Thus run length encoding converts a space complexity for such images from
O(N<sup>2</sup>) to O(N)

Hence I have used runlength encoding to save images generated by microscope. 

# How is run length encoding is working to generate a parasite image

Explaination of algorithm used to generate microscope images

Assumption taken to generate run length encoding:

i) Each row has white color followed by black followed by white([0,0,0....,1,1,1....,0,0,0]). Regions of black in a single row does not repeat itself

Why is this assumption reasonable?

We could still get arbitrary shape of microorganism as there is no such restriction on a column. A column can look like [0,0,0,...1,1,1...0,0,0...1,0,1,1]. Thus this assumption still generates random images.

How did I manage that the parasite is continuos?

  i) On the previous row let's say parasite spans from column number a to b. Then in this block to ensure connectivity I chose a random column number d between a and b. 
  
  ii) Then I randomly chose two column numbers e and f between (0,d) and (d,99999) respectively. These e and f would represent the range of parasite in this row. Since d is in between e and f, this ensures continuity.
  
How did I ensure that parasite occupies atleast 25% of total area?

 In each row I selected two numbers in random which are the start and end of the black parasite in this row. Average distance of two points randomly selected in a line = L/3. If the length of the row is L. 
 
 Total area=L<sup>2</sup> <br>
 25% of total area = L<sup>2</sup>/4 <br>
 average length of parasite in each row = L/3
 Therefore height of the parasite=25% of total area/average length of parasite in each row=3L/4
 
 Therefore I have generated images such that the first row of parasite lie in the first L/8 rows at random and the last row of parasite lie in the last L/8 rows at random. Ensuring that the height of parasite is greater than 3L/4. Hence it covers more than 25 percent of area.

How does the run length encoding of generated image look like?

Run length encoding of a microscope image will be a list:

i) at index =0 in this list we have an integer between 0 to 12499 which represents the row number including and above which all columns are just white

ii) at index=-1 in this list we only have an integer between 87500 to 99999 which represents the row number including and below which all columns are white

iii) all the other rows have the following format:

[(0,index_upto_which this_column_is_white,1),(start_index_of_black,end_index_of_black,0),(index_after_which_this_column_is_white,99999,1)]

In [1]:
def generate_mo(image_size):
  width, height = image_size
  rle = []

  # Choose row numbers for the black segment
  row_start = random.randint(0, 12499)
  row_end = random.randint(87500, 99999)
  rle.append(row_start-1) #line defines row_number which we have white region in the image
  
  # White before row_start
  
  # Random black segment starting at row_start
  a, b = random.sample(range(width), 2)
  a, b = sorted([a, b])  # Ensure a < b

  for row in range(row_start, row_end + 1):
    # White segment before black points
    rle.append([(0,a-1, 1),(a,b,0),(b+1,width-1, 1)])   # Update a and b for next row
    d = random.randint(a, b)
    e = random.randint(0, d)
    f = random.randint(d, width - 1)
    a, b = e, f

  # White after row_end
  rle.append(row_end+1) #line defines row_numberfrom which we have white region in the image 

  return rle

# How is run length encoding working to generate dye images

Explaination of algorithm used to generate microscope images

To generate dye images we will do a run length encoding and our assumption of each row having on black region still holds. 
But since dye can be very less or more in that row. We will bring variablity on the amount of dye. To create multiple test cases. 

How to bring variablity in the amount of dye each row is affected to?

We can do so by keeping a size function which tells us in how many segments, we are going to divide our line it ranges from 2 to 6.

Now for each row, if we divide it in N segments we will color N-1 segment as white and 1 segment as black. Greater N leads to smaller black region.


How does the run length encoding of dye look like?

shape=(100000,9)

Each row will be like this : [(0,index_upto_which this_column_has_no_dye,1),(start_index_of_dye,end_index_of_dye,0),(index_after_which_this_column_has_no_dye,99999,1)]

In [13]:
def generate_dye(image_size):
  width, height = image_size
  rle=[]
  #defines size in how many part we will divide our row
  size=random.randint(2, 6)
  for _ in range(0, height):
    
    if size==2:
      [a,b]=np.random.randint(0,width,size=2)
      rle.append([(0,a-1,1),(a,b,0),(b+1,width-1,1)])
        
    elif size==3:
      [a,b,c]=np.random.randint(0,width,size=3)
      interval_length1=b-a+1
      interval_length2=c-b+1
      min_length = min(interval_length1, interval_length2)
      if min_length==interval_length1:
        rle.append([(0,a-1,1),(a,b,0),(b+1,width-1,1)])
      else:
        rle.append([(0,b-1,1),(b,c,0),(c+1,width-1,1)])
        
    elif size==4:
      [a,b,c,d]=np.random.randint(0,width,size=4)
      a,b,c,d=sorted([a,b,c,d])
      interval_length1=b-a+1
      interval_length2=c-b+1
      interval_length3=d-c+1
      min_length = min(interval_length1, interval_length2, interval_length3)
      if min_length==interval_length1:
        rle.append([(0,a-1,1),(a,b,0),(b+1,width-1,1)])
      elif min_length==interval_length2:
        rle.append([(0,b-1,1),(b,c,0),(c+1,width-1,1)])
      else:
        rle.append([(0,c-1,1),(c,d,0),(d+1,width-1,1)])
        
    elif size==5:
      [a,b,c,d,e]=np.random.randint(0,width,size=5)
      a,b,c,d,e=sorted([a,b,c,d,e])
      interval_length1=b-a+1
      interval_length2=c-b+1
      interval_length3=d-c+1
      interval_length4=e-d+1
      min_length = min(interval_length1, interval_length2, interval_length3,interval_length4)
      if min_length==interval_length1:
        rle.append([(0,a-1,1),(a,b,0),(b+1,width-1,1)])
      elif min_length==interval_length2:
        rle.append([(0,b-1,1),(b,c,0),(c+1,width-1,1)])
      elif min_length==interval_length3:
        rle.append([(0,c-1,1),(c,d,0),(d+1,width-1,1)])
      else:
        rle.append([(0,d-1,1),(d,e,0),(e+1,width-1,1)])
        
    else:
      [a,b,c,d,e,f]=np.random.randint(0,width,size=6)
      a,b,c,d,e,f=sorted([a,b,c,d,e,f])
      interval_length1=b-a+1
      interval_length2=c-b+1
      interval_length3=d-c+1
      interval_length4=e-d+1
      interval_length5=f-e+1
      min_length = min(interval_length1, interval_length2, interval_length3,interval_length4,interval_length5)
      if min_length==interval_length1:
        rle.append([(0,a-1,1),(a,b,0),(b+1,width-1,1)])
      elif min_length==interval_length2:
        rle.append([(0,b-1,1),(b,c,0),(c+1,width-1,1)])
      elif min_length==interval_length3:
        rle.append([(0,c-1,1),(c,d,0),(d+1,width-1,1)])
      elif min_length==interval_length4:
        rle.append([(0,d-1,1),(d,e,0),(e+1,width-1,1)])
      else:
        rle.append([(0,e-1,1),(e,f,0),(f+1,width-1,1)])
  return rle


# How is has_cancer working?

Algorithm to check wheather it has cancer?

This algorithm takes the generated image and takes intersection between parasite and die region row by row and add it to get common area

In [16]:
def get_intersection(range1, range2):
  start1, end1 = range1
  start2, end2 = range2

  # Find the maximum starting point
  intersection_start = max(start1, start2)

  # Find the minimum ending point
  intersection_end = min(end1, end2)

  # Check if there is an actual intersection
  if intersection_start <= intersection_end:
    return intersection_start, intersection_end
  else:
    return -1,-1

In [22]:
def has_cancer(mo_data,dye_data):
    common_area=0
    total_area=0
    for i in range(1,len(mo_data)-1):
      mo_data_index=i
      dye_data_index=i+mo_data[0]
      mo_data_col_start=mo_data[mo_data_index][1][0]
      mo_data_col_end=mo_data[mo_data_index][1][1]
      dye_data_col_start=dye_data[dye_data_index][1][0]
      dye_data_col_end=dye_data[dye_data_index][1][1]
      intersection_col_start,intersection_col_end=get_intersection((mo_data_col_start,mo_data_col_end),(dye_data_col_start,dye_data_col_end))
      if intersection_col_start==-1 or intersection_col_end==-1:
        continue
      common_area=common_area+intersection_col_end-intersection_col_start+1
      total_area=total_area+mo_data_col_end-mo_data_col_start+1
    if total_area ==0 :
      return (total_area,common_area,common_area/total_area,"No")
    else:
      if(common_area/total_area>0.1):
        return (total_area,common_area,common_area/total_area,"Yes")
      else:
        return (total_area,common_area,common_area/total_area,"No")

# Calculation of runtime and storage cost

In [30]:
start_time=time.time()
for _ in range(10):
  image_size = (100000, 100000)
  mo_data=generate_mo(image_size)
  dye_data=generate_dye(image_size)
  total_area,common_area, ratio, result=has_cancer(mo_data,dye_data)
  print(f"Total area covered by microorganism = {total_area}")
  print(f"common area between the microorganism and dye = {common_area}")
  print(f"percentage of common area = {ratio*100} %")
  print(f"has cancer : {result}")
  print()
end_time=time.time()

print(f"Total time from generating 10 sample(parasite + dye) to check whether it has cancer ={end_time-start_time} sec")
print(f"average time from generating 1 sample (parasite + dye) to check whether it has cancer = {(end_time-start_time)/10} sec")
print(f"average memory used in 1 sample (parasite+dye) = {Memory_used_in_megabytes/10} Mb" )

Total area covered by microorganism = 2963297350
common area between the microorganism and dye = 197110688
percentage of common area = 6.6517350342853705 %
has cancer : No

Total area covered by microorganism = 2862331845
common area between the microorganism and dye = 190303966
percentage of common area = 6.648564048659425 %
has cancer : No

Total area covered by microorganism = 2892040720
common area between the microorganism and dye = 132497214
percentage of common area = 4.581443583546776 %
has cancer : No

Total area covered by microorganism = 2933052299
common area between the microorganism and dye = 136000871
percentage of common area = 4.636837571780373 %
has cancer : No

Total area covered by microorganism = 2859284948
common area between the microorganism and dye = 295892150
percentage of common area = 10.348466675452173 %
has cancer : Yes

Total area covered by microorganism = 1912133768
common area between the microorganism and dye = 884058618
percentage of common area = 46

In [29]:
Memory_used_in_megabytes=0
for _ in range(10):
  image_size = (100000, 100000)
  mo_data=generate_mo(image_size)
  dye_data=generate_dye(image_size)
  mo_usage_megabytes = asizeof.asizeof(mo_data)/(1024 ** 2)
  dye_usage_megabytes = asizeof.asizeof(mo_data)/(1024 ** 2)
  Memory_used_in_megabytes+=(mo_usage_megabytes+dye_usage_megabytes)
print(f"average memory used in 1 sample (parasite+dye) = {Memory_used_in_megabytes/10} Mb" )

average memory used in 1 sample (parasite+dye) = 74.41591186523438 Mb


As can be seen above Runtime for one image is approximately 4 to 5 seconds from generation to checking whether it has parasite or not.

Memory taken by each image is in the range of 70 - 80 mbs as can be seen above


# Other compression types 

i) Another encoding idea that came in my mind and could be used as the image only has 1s and 0s (black and white) therefore we can take a row [1 0 0 0 1 0 1...] we can take this row and think of it as a binary number and just change it to higher base.

Eg [1 0 0 0 0 0 0 0 0 0] this row can just be represented as 1024 in decimal. But if we did this encoding computing area would have been complex as we have to unpack it to calculate area

# Tools used

i) Jupyter notebook and vs code to write and test code <br>
ii) chat gpt, gemini and perplexity to help me write basic function such as get intersection and has_cancer <br>
iii) stack overflow to help me construct an algorithm to generate continuous image of parasite randomly